Brain MRI Segmentation

In [ ]:
#pip install d2l==1.0.0a1.post0

Import:

In [ ]:
# Import
from glob import glob
from d2l import tensorflow as d2l
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras.layers as layer
from PIL import Image
# For GPU
from numba import jit,cuda

Creazione di una specifica struttura del dataset per UNeXt

In [ ]:
# Mi serve per salvare il path delle immagini
img = []

# Percorso per prendere tutte le maschere e non le immagini
msk = glob('C:/Users/MATTEO/Desktop/Programmi/Python/DeepLearning/ProgettoDeep/Brain_MRI/*/*_mask*')

# Scorro le maschere e rimuovo mask per la corrispondente immagine associata
for i in msk:
    img.append(i.replace('_mask',''))


for i in range(len(msk)):
    # sposto immagine
    im=Image.open(img[i])
    ms=Image.open(msk[i])
    im.save("C:\\Users\\MATTEO\\Desktop\\Programmi\\Python\\DeepLearning\\ProgettoDeep\\DsDL\\images\\prova"+str(i)+".tif")
    ms.save("C:\\Users\\MATTEO\\Desktop\\Programmi\\Python\\DeepLearning\\ProgettoDeep\\DsDL\\masks\\0\\prova"+str(i)+".tif")

Caricamento dei percorsi delle immagini per UNET e ResUnet:

In [ ]:
def load_path():
    print("Load path..")

    # Mi serve per salvare il path delle immagini
    image_files = []

    # Percorso per prendere tutte le maschere e non le immagini
    image_masks = glob('C:/Users/MATTEO/Desktop/Programmi/Python/DeepLearning/ProgettoDeep/Brain_MRI/*/*_mask*')

    # Scorro le maschere e rimuovo mask per la corrispondente immagine associata
    for i in image_masks:
        image_files.append(i.replace('_mask',''))
        
    return image_files,image_masks

Caricamento delle immagini + Data augmentation:

In [ ]:
def load_img():
    # prendo tutti i path di immagini
    img,msk=load_path()
    print("Load image..")

    # np array per immgini e maschere
    imlst=np.zeros((len(img*4),256,256,3),dtype=np.uint8)
    mslst=np.zeros((len(msk*4),256,256),dtype=np.bool_)
    
    # scorro i path e prendo immagini
    index=0
    j=0
    with d2l.try_gpu():
        for i in range(len(img)):
            index=index+1
            print(index)

            # carico immagine
            im=Image.open(img[i])

            # Data augmentation
            imRotate=im.rotate(90)
            imFlippedLR=im.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
            imFlippedTB=im.transpose(Image.Transpose.FLIP_TOP_BOTTOM)

            # carico maschera
            ms=Image.open(msk[i])
            msRotate=ms.rotate(90)
            msFlippedLR=ms.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
            msFlippedTB=ms.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
            
            # metto nell'nparray
            imlst[j]=im
            mslst[j]=ms
            j=j+1
            imlst[j]=imRotate
            mslst[j]=msRotate
            j=j+1
            imlst[j]=imFlippedLR
            mslst[j]=msFlippedLR
            j=j+1
            imlst[j]=imFlippedTB
            mslst[j]=msFlippedTB
            j=j+1
    print(j)
    return imlst,mslst


Training, test, validation split:

In [ ]:
def split_data():
    img,msk=load_img()
    
    print("Spleet Data..")
    # Split dataset in train 80% e test 20%
    imgTrain,imgTest,mskTrain, mskTest = train_test_split(img,msk, test_size = 0.25)
    # Split dal train il validation il 20% 
    # train, validation = train_test_split(train, test_size = 0.25)

    #return train,test,validation
    return imgTrain,mskTrain, imgTest, mskTest

Reshape:

In [ ]:
#train,test,validation=split_data()
imgTrain,mskTrain, imgTest, mskTest=split_data()
mskTrain=mskTrain.reshape((mskTrain.shape[0],256,256,1))
imgTrain=imgTrain.reshape((imgTrain.shape[0],256,256,3))
print(imgTrain.shape)
print(mskTrain.shape)

Unet:

 Il problema era innanzitutto nello shape delle maskere perche io pensavo che (None,256,256) fosse uguale a (None,256,256,1) invece no.
 Inoltre nella segmentazione la rete deve generare in output non 1 valore come nelle MLP, ma una immagine di uguale dimensione a quella in input
 se ho 256,256,3 in input, in output deve avere per funzionare 256,256,1

Re-implementazione Unet per capire la propagazione dell'immagine con i vari re-size dovuti alla convoluzione ed al pooling

In [ ]:
class Unet:
    def __init__(self):

        inputs = tf.keras.layers.Input((256, 256, 3))

        # Encoder
        c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
        #c1 = tf.keras.layers.Dropout(0.1)(c1)
        c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
        #b1 = tf.keras.layers.BatchNormalization()(c1)
        r1 = tf.keras.layers.ReLU()(c1)
        p1 = tf.keras.layers.MaxPooling2D((2, 2))(r1)

        c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
        #c2 = tf.keras.layers.Dropout(0.1)(c2)
        c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
        #b2 = tf.keras.layers.BatchNormalization()(c2)
        r2 = tf.keras.layers.ReLU()(c2)
        p2 = tf.keras.layers.MaxPooling2D((2, 2))(r2)
        
        c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
        #c3 = tf.keras.layers.Dropout(0.2)(c3)
        c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
        #c3 = tf.keras.layers.BatchNormalization()(c3)
        r3 = tf.keras.layers.ReLU()(c3)
        p3 = tf.keras.layers.MaxPooling2D((2, 2))(r3)
        
        c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
        #c4 = tf.keras.layers.Dropout(0.2)(c4)
        c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
        #b4 = tf.keras.layers.BatchNormalization()(c4)
        r4 = tf.keras.layers.ReLU()(c4)
        p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(r4)

        # Bridge
        c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
        c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
        u5 = tf.keras.layers.BatchNormalization()(c5)
        u5 = tf.keras.layers.ReLU()(u5)
        #c5 = tf.keras.layers.Dropout(0.3)(r5)
        

        # Decoder 
        u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(u5)
        u6 = tf.keras.layers.concatenate([u6, r4])
        c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
        c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)
        #u6 = tf.keras.layers.BatchNormalization()(c6)
        u6 = tf.keras.layers.ReLU()(c6)

        
        u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(u6)
        u7 = tf.keras.layers.concatenate([u7, r3])
        c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
        c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)
        #u7 = tf.keras.layers.BatchNormalization()(c7)
        u7 = tf.keras.layers.ReLU()(c7)

        
        u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u7)
        u8 = tf.keras.layers.concatenate([u8, r2])
        c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
        c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)
        #u8 = tf.keras.layers.BatchNormalization()(c8)
        u8 = tf.keras.layers.ReLU()(c8)
        
        u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u8)
        u9 = tf.keras.layers.concatenate([u9, r1], axis=3)
        c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
        c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
        #u9 = tf.keras.layers.BatchNormalization()(c9)
        u9 = tf.keras.layers.ReLU()(c9)

        # Output
        outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(u9)
        
        self.model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # For gpu
    @jit()
    def training(self,xtrain,ytrain):
        self.model.fit(xtrain,ytrain,batch_size=32,epochs=1)

In [ ]:
with d2l.try_gpu():
    UNet=Unet()
    UNet.model.summary()
    UNet.training(imgTrain,mskTrain)
    

Re-implementazione della ResUnet:

In [ ]:
class ResUnet:
    def __init__(self):
        inputs = tf.keras.layers.Input((256,256,3))

        # Encoder
        a1 = tf.keras.layers.Conv2D(64,1,activation= "relu", padding= "same")(inputs)

        e1 = tf.keras.layers.Conv2D(64,3,activation= "relu", padding= "same")(inputs)
        b1 = tf.keras.layers.BatchNormalization()(e1)
        r1 = tf.keras.layers.ReLU()(b1)
        e1 = tf.keras.layers.Conv2D(64,3,activation= "relu", padding= "same")(r1)
        e1 = tf.keras.layers.add([e1,a1])
        m1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(e1)


        a2 = tf.keras.layers.Conv2D(128,1,activation= "relu", padding= "same")(m1)
        
        b2 = tf.keras.layers.BatchNormalization()(m1)
        r2 = tf.keras.layers.ReLU()(b2)
        e2 = tf.keras.layers.Conv2D(128,3,activation= "relu", padding= "same")(r2)
        b2 = tf.keras.layers.BatchNormalization()(e2)
        r2 = tf.keras.layers.ReLU()(b2)
        e2 = tf.keras.layers.Conv2D(128,3,activation= "relu", padding= "same")(r2)
        e2 = tf.keras.layers.add([a2,e2])
        m2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(e2)


        a3 = tf.keras.layers.Conv2D(256,1,activation= "relu", padding= "same")(m2)

        b3 = tf.keras.layers.BatchNormalization()(m2)
        r3 = tf.keras.layers.ReLU()(b3)
        e3 = tf.keras.layers.Conv2D(256,3,activation= "relu", padding= "same")(r3)
        b3 = tf.keras.layers.BatchNormalization()(e3)
        r3 = tf.keras.layers.ReLU()(b3)
        e3 = tf.keras.layers.Conv2D(256,3,activation= "relu", padding= "same")(r3)
        e3 = tf.keras.layers.add([a3,e3])
        m3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(e3)


        # Bridge
        a4 = tf.keras.layers.Conv2D(512,1,activation= "relu", padding= "same")(m3)

        b4 = tf.keras.layers.BatchNormalization()(m3)
        r4 = tf.keras.layers.ReLU()(b4)
        e4 = tf.keras.layers.Conv2D(512,3,activation= "relu", padding= "same")(r4)
        b4 = tf.keras.layers.BatchNormalization()(e4)
        r4 = tf.keras.layers.ReLU()(b4)
        e4 = tf.keras.layers.Conv2D(512,3,activation= "relu", padding= "same")(r4)
        e4 = tf.keras.layers.add([a4,e4])


        # Decoder
        d5 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(e4)
        d5 = tf.keras.layers.concatenate([e3,d5])

        b6 = tf.keras.layers.BatchNormalization()(d5)
        r6 = tf.keras.layers.ReLU()(b6)
        d6 = tf.keras.layers.Conv2D(512,3,activation= "relu", padding= "same")(r6)
        b6 = tf.keras.layers.BatchNormalization()(d6)
        r6 = tf.keras.layers.ReLU()(b6)
        d6 = tf.keras.layers.Conv2D(512,3,activation= "relu", padding= "same")(r6)
        d6 = tf.keras.layers.add([d5,d6])


        d7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(d6)
        d7 = tf.keras.layers.concatenate([d7,e2])

        b8 = tf.keras.layers.BatchNormalization()(d7)
        r8 = tf.keras.layers.ReLU()(b8)
        d8 = tf.keras.layers.Conv2D(256,3,activation= "relu", padding= "same")(r8)
        b8 = tf.keras.layers.BatchNormalization()(d8)
        r8 = tf.keras.layers.ReLU()(b8)
        d8 = tf.keras.layers.Conv2D(256,3,activation= "relu", padding= "same")(r8)
        d8 = tf.keras.layers.add([d7,d8])
        

        d9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(d8)
        d9 = tf.keras.layers.concatenate([d9,e1])
        
        b10 = tf.keras.layers.BatchNormalization()(d9)
        r10 = tf.keras.layers.ReLU()(b10)
        d10 = tf.keras.layers.Conv2D(128,3,activation= "relu", padding= "same")(r10)
        b10 = tf.keras.layers.BatchNormalization()(d10)
        r10 = tf.keras.layers.ReLU()(b10)
        d10 = tf.keras.layers.Conv2D(128,3,activation= "relu", padding= "same")(d10)
        d10 = tf.keras.layers.add([d9,d10])


        # Output
        outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(d10)

        self.model = tf.keras.Model(inputs = [inputs], outputs = [outputs])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # For gpu
    @jit()
    def training(self,xtrain,ytrain):
        self.model.fit(xtrain,ytrain,batch_size=32,epochs=1)

Training e valutazione modello

In [ ]:
with d2l.try_gpu():
    ResUnet=ResUnet()
    ResUnet.model.summary()
    ResUnet.training(imgTrain,mskTrain)
    

In [ ]:
with d2l.try_gpu():
    UNet.model.evaluate(imgTest,mskTest)